### **GridSearchCV (HyperTuning) on HouseLoan Dataset**

* GridSearchCV is a method in machine learning used to optimize the performance of a model by systematically testing different combinations of hyperparameters. It performs an exhaustive search over a specified parameter grid, where each combination of hyperparameters is evaluated using cross-validation to determine the best performing set. This technique helps in identifying the most optimal set of parameters to improve the accuracy and generalization of the model.

**Step 1: Import Libraries**


In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV


**Step 2: Load the Dataset**

In [2]:
df = pd.read_csv("E:\\Machine Learning\\Datasets\\loan.csv")

In [3]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
df.shape

(614, 13)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


**Step 3: Data Preprocessing**

In [6]:
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [7]:
df['Gender'].mode()
df['Gender'] = df['Gender'].fillna('Male')

In [8]:
df['Married'] = df['Gender'].fillna(df['Married'].mode())

In [9]:
df['LoanAmount'] = df['LoanAmount'].fillna(df['LoanAmount'].median())

In [10]:
df['Loan_Amount_Term'].median()
df['Loan_Amount_Term'] = df['Loan_Amount_Term'].fillna(360.0)

In [11]:
df['Credit_History'].mode()
df['Credit_History'] = df['Credit_History'].fillna(1.0)

In [12]:
df['Dependents'].mode()
df['Dependents'] = df['Dependents'].fillna(0)

In [13]:
df['Self_Employed'].mode()
df['Self_Employed'] = df['Self_Employed'].fillna(0)

In [14]:
df.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [15]:
df.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [16]:
df_cat = df.select_dtypes(include = ['object'])
df_cat.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'Property_Area', 'Loan_Status'],
      dtype='object')

**Step 4 : Encoding the Categorical columns**

In [17]:
columns_to_encode = ['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'Property_Area', 'Loan_Status']

In [18]:
# Convert columns to string type to handle mixed data types
for column in columns_to_encode:
    df[column] = df[column].astype(str)

# Now apply LabelEncoder
label_encoder = LabelEncoder()

for column in columns_to_encode:
    df[column] = label_encoder.fit_transform(df[column])

# Check the result
df.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0,1,1,0,0,1,5849,0.0,128.0,360.0,1.0,2,1
1,1,1,1,1,0,1,4583,1508.0,128.0,360.0,1.0,0,0
2,2,1,1,0,0,2,3000,0.0,66.0,360.0,1.0,2,1
3,3,1,1,0,1,1,2583,2358.0,120.0,360.0,1.0,2,1
4,4,1,1,0,0,1,6000,0.0,141.0,360.0,1.0,2,1


**Step 5: Split the Data into Training and Testing Sets**


In [19]:
# Select features (X) and target (y)
X = df[['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education', 
        'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 
        'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area']]

# Target variable (y)
y = df['Loan_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


**Step 6: Train the Random Forest Classifier**

In [20]:
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

**Step 7: Make Predictions and Evaluate the Model**

In [21]:
y_pred = rf_clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)


In [22]:
accuracy

0.7642276422764228

In [26]:
from sklearn.metrics import confusion_matrix

# Define the hyperparameter grid for Random Forest Classifier
param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10],
    'criterion': ['gini', 'entropy']
}

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5)

# Perform grid search
grid_search.fit(X_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:")
print(grid_search.best_params_)

# Print the best score
print("Best Score:")
print(grid_search.best_score_)

# Evaluate the best model on the test set
y_pred = grid_search.best_estimator_.predict(X_test)

# Print the accuracy score
print("Accuracy Score:")
print(accuracy_score(y_test, y_pred))

# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Best Hyperparameters:
{'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
Best Score:
0.8206349206349207
Accuracy Score:
0.7560975609756098
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.42      0.55        43
           1       0.75      0.94      0.83        80

    accuracy                           0.76       123
   macro avg       0.77      0.68      0.69       123
weighted avg       0.76      0.76      0.73       123

Confusion Matrix:
[[18 25]
 [ 5 75]]


C:\Users\gkris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
